In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display as ipd
import librosa
import librosa.display
import scipy.io.wavfile as wavf

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from datetime import datetime
from keras.preprocessing.sequence import pad_sequences

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display as ipd
import librosa
import librosa.display
from tqdm import tqdm
import os

In [3]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras.models import load_model
import pickle

In [23]:
def predict(chunk_data,classes):
    
    df = pd.read_csv(r"references.csv")
    print("classes = ", classes)

    def features_extractor(file):
        audio,sample_rate = librosa.load(file , res_type = 'kaiser_fast')
        mfccs_features = librosa.feature.mfcc(y = audio,sr = sample_rate, n_mfcc=40)
        mfccs_scaled_features = np.mean(mfccs_features.T,axis = 0)
        return mfccs_scaled_features
    
    extracted_features = []
    #classes = ["plumbing","iro","service","when"]
    #audio_path_dir =str("C:Users\sai\Desktop\Capstone project\DL\coding\audio_data")
    cwd = os.getcwd()
    for i in df.index:
        class_name = df['class_name'][i]

        audio_file_name = str(df['file_name'][i])

        if class_name in classes:
            #audio_path = cwd + "\\audio_data\\" + audio_file_name + ".wav"
            audio_path = "D:\PES\Capstone\phase two" + "\wav\\" +   audio_file_name
            #print(audio_path)
            data = features_extractor(audio_path)
            extracted_features.append([class_name,data])
            
    df_extracted_features = pd.DataFrame(extracted_features,columns = ['class_name' , 'feature_vector'])
    
    mini_df = df_extracted_features
    
    X = np.array(mini_df['feature_vector'].tolist())
    y = np.array(mini_df['class_name'].tolist())
    
    #y = np.array(pd.get_dummies(y))
    labelencoder=LabelEncoder()
    print('Y before',y)
    y=labelencoder.fit_transform(y)
    print('Y integer',y)
    onehot_encoder = OneHotEncoder(sparse=False)
    y = y.reshape(len(y), 1)
    y = onehot_encoder.fit_transform(y)
#     y=to_categorical(labelencoder.fit_transform(y))
#     labelencoder.fit(y)
#     y=labelencoder.transform(y)
#     y=np.array([[i]for i in y])
    print('Y after',y)
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 0)
#     print(X_train.shape,y_train.shape,X_test.shape,y_test.shape)
    
    
    
    num_labels=y.shape[1]
    model=Sequential()
    ###first layer
    model.add(Dense(100,input_shape=(40,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    ###second layer
    model.add(Dense(200))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    ###third layer
    model.add(Dense(100))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))

    ###final layer
    model.add(Dense(num_labels))
    model.add(Activation('softmax'))
    
    model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')
    from tensorflow.keras.callbacks import ModelCheckpoint
    from datetime import datetime 

    num_epochs = 500
    num_batch_size = 4

    model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), verbose=0)
    test_accuracy=model.evaluate(X_test,y_test,verbose=0)
    print("Test Accuracy",test_accuracy[1])
    
    
    #Preditcting the class
    audio,sample_rate = librosa.load('test/nanna_test.wav' , res_type = 'kaiser_fast')
    mfccs_features = librosa.feature.mfcc(y = audio , sr=22050, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    
    mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
    
    predicted_label = model.predict_classes(mfccs_scaled_features)
    
    print("Predcited Label",predicted_label)
    prediction_class = labelencoder.inverse_transform(predicted_label) 
    print("Predcited Class",prediction_class[0])
    return prediction_class[0]

    
    
    
    

In [ ]:
output=predict([],['what', 'hathira', 'hathiradha', 'are', 'nanna'])

classes =  ['what', 'hathira', 'hathiradha', 'are', 'nanna']
Y before ['are' 'are' 'are' 'are' 'are' 'hathiradha' 'hathiradha' 'hathiradha'
 'hathiradha' 'hathiradha' 'hathira' 'hathira' 'hathira' 'hathira'
 'hathira' 'nanna' 'nanna' 'nanna' 'nanna' 'nanna' 'what' 'what' 'what'
 'what' 'what' 'are' 'are' 'are' 'are' 'are' 'hathiradha' 'hathiradha'
 'hathiradha' 'hathiradha' 'hathiradha' 'hathira' 'hathira' 'hathira'
 'hathira' 'hathira' 'nanna' 'nanna' 'nanna' 'nanna' 'nanna' 'what' 'what'
 'what' 'what' 'what' 'are' 'are' 'are' 'are' 'are' 'hathiradha'
 'hathiradha' 'hathiradha' 'hathiradha' 'hathiradha' 'hathira' 'hathira'
 'hathira' 'hathira' 'hathira' 'nanna' 'nanna' 'nanna' 'nanna' 'nanna'
 'what' 'what' 'what' 'what' 'what']
Y integer [0 0 0 0 0 2 2 2 2 2 1 1 1 1 1 3 3 3 3 3 4 4 4 4 4 0 0 0 0 0 2 2 2 2 2 1 1
 1 1 1 3 3 3 3 3 4 4 4 4 4 0 0 0 0 0 2 2 2 2 2 1 1 1 1 1 3 3 3 3 3 4 4 4 4
 4]
Y after [[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0